# Converting a List of DICOMs to PNG. This notebook will read DICOM files and convert them to 16-bit PNG. Flipping logic is included such that both left and right breast images will be pointed in the same direction to improve model training.

In [2]:
import pydicom
import numpy as np
import pandas as pd
import png
from pathlib import Path

In [3]:
# Get DICOM image metadata
class DCM_Tags():
    def __init__(self, img_dcm):
        try:
            self.laterality = img_dcm.ImageLaterality
        except AttributeError:
            self.laterality = np.nan
            
        try:
            self.view = img_dcm.ViewPosition
        except AttributeError:
            self.view = np.nan
            
        try:
            self.orientation = img_dcm.PatientOrientation
        except AttributeError:
            self.orientation = np.nan

# Check whether DICOM should be flipped
def check_dcm(imgdcm):
    # Get DICOM metadata
    tags = DCM_Tags(imgdcm)
    
    # If image orientation tag is defined
    if ~pd.isnull(tags.orientation):
        # CC view
        if tags.view == 'CC':
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & (tags.orientation[1] == 'L'):
                flipVert = True
            elif (tags.laterality == 'R') & (tags.orientation[1] == 'R'):
                flipVert = True
            else:
                flipVert = False
        
        # MLO or ML views
        elif (tags.view == 'MLO') | (tags.view == 'ML'):
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HL')):
                flipVert = True
            elif (tags.laterality == 'R') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HR')):
                flipVert = True
            else:
                flipVert = False
        
        # Unrecognized view
        else:
            flipHorz = False
            flipVert = False
            
    # If image orientation tag is undefined
    else:
        # Flip RCC, RML, and RMLO images
        if (tags.laterality == 'R') & ((tags.view == 'CC') | (tags.view == 'ML') | (tags.view == 'MLO')):
            flipHorz = True
            flipVert = False
        else:
            flipHorz = False
            flipVert = False
            
    return flipHorz, flipVert

# Save DICOM pixel array as PNG
def save_dcm_image_as_png(image, png_filename, bitdepth=12):
    with open(png_filename, 'wb') as f:
        writer = png.Writer(height=image.shape[0], 
                            width=image.shape[1], 
                            bitdepth=bitdepth, 
                            greyscale=True)
        writer.write(f, image.tolist())

In [4]:
# Convert list of DICOMs to PNGs
def process_dcm_list(dcm_list, save_path):
    for i, dcm_path in enumerate(dcm_list):
        print(f"Processing DICOM #{i}...")
        
        # Load DICOM
        dcm = pydicom.dcmread(dcm_path)
        img = dcm.pixel_array
        
        # Check if a horizontal flip is necessary
        horz, _ = check_dcm(dcm)
        if horz:
            # Flip img horizontally
            img = np.fliplr(img)
        
        # Get new file name
        split_fn = dcm_path[:-4].split('/')
        new_fn = f"{split_fn[-1]}_conv.png"
        
        # Save PNG
        png_path = save_path + new_fn
        save_dcm_image_as_png(img, png_path)

In [5]:
image_path = "/media/szelesteya/F824D4D024D492CC/EMBED-images/pleomorphic-calcification"

# Provide a list of DICOM paths and a target directory
dcm_list = []

with open('/home/szelesteya/projects/EMBED_Open_Data/pleomorphic_path.csv', 'r') as f:
    header_skipped = False
    
    for line in f:
        if not header_skipped:
            header_skipped = True
            continue
            
        path = image_path + '/' + '/'.join(line.split('/')[-3:]).rstrip('\n')

        if Path(path).exists():
            dcm_list.append(path)
        
save_path = image_path + '-png'

# Convert DICOMs
process_dcm_list(dcm_list, save_path)

Processing DICOM #0...
Processing DICOM #1...
Processing DICOM #2...
Processing DICOM #3...
Processing DICOM #4...
Processing DICOM #5...
Processing DICOM #6...
Processing DICOM #7...
Processing DICOM #8...
Processing DICOM #9...
Processing DICOM #10...
Processing DICOM #11...
Processing DICOM #12...
Processing DICOM #13...
Processing DICOM #14...
Processing DICOM #15...
Processing DICOM #16...
Processing DICOM #17...
Processing DICOM #18...
Processing DICOM #19...
Processing DICOM #20...
Processing DICOM #21...
Processing DICOM #22...
Processing DICOM #23...
Processing DICOM #24...
Processing DICOM #25...
Processing DICOM #26...
Processing DICOM #27...
Processing DICOM #28...
Processing DICOM #29...
Processing DICOM #30...
Processing DICOM #31...
Processing DICOM #32...
Processing DICOM #33...
Processing DICOM #34...
Processing DICOM #35...
Processing DICOM #36...
Processing DICOM #37...
Processing DICOM #38...
Processing DICOM #39...
Processing DICOM #40...
Processing DICOM #41...
Pr